# Segmenting and Clustering Neighborhoods in Toronto

In [4]:
# imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Datascraping

In the below code snippet I request the webpage from the url, then I parse it to a BeautifulSoup object to easily find the table I need. Then I read the table to a Pandas Dataframe, but since the read_html returns a list of dataframes, I chose the first (and only) dataframe in the list.

In [26]:
# scrape data from website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

# extract table from url raw data
bs = BeautifulSoup(response.text, 'html.parser')
table = bs.find('table', {'class': 'wikitable'})

# convert table to dataframe
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Preprocessing

The below code snippet preprocesses the dataframe. First I remove all rows where no borough is assigned. Then, all rows with no assigned neighborhood gets assigned the borough

In [32]:
# preprocessing
df = df[df['Borough'] != 'Not assigned']
df['Neighbourhood'][df['Neighbourhood'] == 'Not assigned'] == df['Borough'][df['Neighbourhood'] == 'Not assigned']
df = df.groupby('Postal Code', as_index=False).agg({'Borough': 'first','Neighbourhood': ' '.join})
df.head(103)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [33]:
df.shape

(103, 3)